In [2]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root="data", # 데이터를 저장할 root 디렉토리
    train=True, # 훈련용 데이터 설정
    download=True, # 다운로드
    transform=ToTensor() # 이미지 변환. 여기서는 TorchTesnor로 변환시킵니다.
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(
    train_data, batch_size=64, shuffle=True
)

test_dataloader = DataLoader(
    test_data, batch_size=64, shuffle=False
)

# PyTorch Modeling
파이토치는 대부분 클래스 기반 모델링을 수행합니다. `torch.nn.Module` 클래스를 상속 받아 만들게 됩니다. 필수적으로 오버라이딩 해야 하는 메소드는 생성자 `__init__`과 순전파를 담당하는 `forward` 입니다.

In [4]:
from torch import nn

class NeuralNetwork(nn.Module):

  def __init__(self):
    # Subclass인 NeuralNetwork의 생성자.
    #   여기에서 상위 클래스인 nn.Module의 생성에 대한 책임을 져야 한다.
    #   책임이란? 부모클래스의 생성자에 필요한 파라미터
    super(NeuralNetwork, self).__init__()

    # 생성자에는 항상 레이어의 구성을 정의
    self.flatten = nn.Flatten() # 입력되는 데이터를 평탄화 시키는 레이어

    # nn.Sequential을 이용해 연속되는 레이어의 구조를 구성
    self.fcl_stack = nn.Sequential(
      # 1층 구성
      nn.Linear(28*28, 128),
      nn.ReLU(),

      # 2층 (출력층)
      nn.Linear(128, 10)
    )
    # Softmax를 따로 쓰지 않는 이유는 실제 모델 순전파 시에 넣어도 상관 없기 때문 ! (훈련 할 떄)
    # 꼭 없어야 하는건 아님 !! 상황에 따라서 넣어 줄 수도 있음

  def forward(self, x):
    # forward에는 입력 데이터 x가 들어온다. 이 때 x의 shape은? (N, 1, 28, 28)
    x = self.flatten(x) # flatten : 평탄화
    y = self.fcl_stack(x)

    return y

# 모델 생성
파이토치를 이용해 모델 객체를 만들고 나서 어떤 장치(device) 환경에서 훈련이나 추론을 수행할지 결정지어줘야 합니다.

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
# 모델을 만들고, 만든 모델을 설정한 환경(device)로 옮긴다는 개념

In [7]:
# 모델을 만들고, 만든 모델을 설정한 환경(device)로 옮긴다는 개념
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fcl_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [8]:
# 짱짱 중요함

# 손실 함수 정의.
loss_fn = nn.CrossEntropyLoss()

# 최적화 기법 정의 (Opitmizer)
#   강사하강법을 수행하기 위한 함수. 경사하강법은 어디에 수행하죠? 모델의 W, b에 수행
#   W, b는 다른 말로 모델의 파라미터(Model Parameter). model.parameters()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [13]:
# 훈련 과정 ( 훈련 루프 정의 )
#   1. DataLoader에서 배치 크기 만큼 데이터를 꺼낸다.
#   2. 데이터를 모델에 통과시킨다. (순전파를 통한 추론 - prediction(inference))
#   3. 순전파(예측)가 끝나면 예측 값이 나오게 되고, 이를 토대로 Loss를 계산
#   4. 역전파를 통한 미분값 계산
#   5. 얻어낸 미분값으로 경사하강법 수행 ( 최적화 )
def train_loop(dataloader, model, loss_fn, optimizer) :
    # 데이터 로더에 있는 데이터 세트의 전체 개수 가져오기
    size = len(dataloader.dataset)

    # 중요!! model을 훈련 모드로 설정. 가중치 갱신이 가능한 상황으로 바꿔주기
    model.train()

    # 1. 데이터 꺼내기. for문을 사용하면 자동으로 nex(iter(dataloader))가 실행된다.
    for batch_idx, (X, y) in enumerate(dataloader) :
        # 현재 데이터 로더에 있는 데이터는 cpu에 존재하고 있기 때문에, 이 데이터들은 gpu로 옮겨야 한다.
        #   모델이 위치한 곳과, 데이터가 위치한 곳을 동일하게 맞춰준다.
        X, y = X.to(device), y.to(device)

        # 2. 순전파 수행
        pred = model(X) # 자동으로 forward가 실행된다.

        # 3. Loss 계산
        loss = loss_fn(pred, y) # 이 때 자동으로 Softmax가 적용된다.

        # 4. 역전파 수행 ( 역전파를 수행하면서 각 가중치의 미분값을 얻어낸다.)
        optimizer.zero_grad() # 최적화 함수안에 남아있는 기존의 기울기를 제거
                              # -> 이전 배치의 기울기가 남아있으면 정확한 기울기를 구하는데 방해가 된다.
        loss.backward() # 역전파 수행. Loss가 Leaf

        # 5. 최적화 수행(경사하강법 수행)
        optimizer.step() # backward()로 구한 미분값을 토대로 최적화

        if batch_idx % 100 == 0 :
            loss, current = loss.item(), batch_idx * len(X)
            print(f"Train Loss : {loss:>7} [{current:>5d} / {size:>5d}]")

In [14]:
# 추론에서는 최적화할 필요가 없다.
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)

  # loss는 배치 별로 계산, correct는 전체 데이터 세트에 대한 평균 정확도
  test_loss, correct = 0, 0

  # 모델을 추론 모드로 바꿔준다.
  model.eval()

  # 추론 과정에서는 기울기를 구할 필요가 없습니다.
  #   따라서 모든 파라미터(model.parameters())의 required_grad=False
  with torch.no_grad():
    # 1. 데이터 꺼내기
    for X, y in dataloader:
      # 데이터를 gpu로 옮겨주기
      #   모델과 같은 장치(device)로 옮겨주기
      X, y = X.to(device), y.to(device)

      # 2. 순전파 수행
      pred = model(X)

      # 3. loss 및 accuracy 계산
      test_loss += loss_fn(pred, y).item()

      # 10개의 예측 값중 가장 큰 곳의 인덱스를 argmax로 찾고, 타겟(y)와 일치하는지 확인
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  # 배치 개수 구하기
  num_batches = len(dataloader)

  # Loss 평균 구하기
  test_loss = test_loss / num_batches

  # Accuracy 구하기
  correct /= size

  print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n")

In [15]:
epochs = 10

for i in range(epochs):
  print(f"Epoch {i+1}\n.....................")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("훈련 끝!")

Epoch 1
.....................
Train Loss : 2.297231435775757 [    0 / 60000]
Train Loss : 1.4752072095870972 [ 6400 / 60000]
Train Loss : 1.0246914625167847 [12800 / 60000]
Train Loss : 0.9473208785057068 [19200 / 60000]
Train Loss : 0.6989604830741882 [25600 / 60000]
Train Loss : 0.8133335709571838 [32000 / 60000]
Train Loss : 0.7934366464614868 [38400 / 60000]
Train Loss : 0.5299373865127563 [44800 / 60000]
Train Loss : 0.6296875476837158 [51200 / 60000]
Train Loss : 0.5670382976531982 [57600 / 60000]
Test Error : 
 Accuracy : 78.8%, Avg Loss : 0.625677

Epoch 2
.....................
Train Loss : 0.7369483113288879 [    0 / 60000]
Train Loss : 0.5683687925338745 [ 6400 / 60000]
Train Loss : 0.5064662098884583 [12800 / 60000]
Train Loss : 0.5606881380081177 [19200 / 60000]
Train Loss : 0.5195249319076538 [25600 / 60000]
Train Loss : 0.6281054615974426 [32000 / 60000]
Train Loss : 0.6380984783172607 [38400 / 60000]
Train Loss : 0.6731709241867065 [44800 / 60000]
Train Loss : 0.61159682

# 훈련된 모델의 가중치를 저장 / 불러오기
불러올 곳에서 **모델의 구조를 알고 있는 경우** 가중치만 저장하면 적은 용량으로 저장하고 불러오는 것이 가능합니다.

In [16]:
# 모델의 구조를 알고 있는 경우( 모델의 클래스를 알고 있는 경우 )
model.state_dict()

OrderedDict([('fcl_stack.0.weight',
              tensor([[-0.0292,  0.0073, -0.0116,  ..., -0.0264, -0.0462, -0.0302],
                      [-0.0158,  0.0146, -0.0059,  ..., -0.0577, -0.1042, -0.0069],
                      [-0.0242, -0.0517, -0.0524,  ..., -0.0679, -0.0490, -0.0611],
                      ...,
                      [-0.0482, -0.0356, -0.0853,  ..., -0.0545,  0.0269,  0.0081],
                      [ 0.0758, -0.0496,  0.0137,  ..., -0.0658, -0.0466,  0.0087],
                      [-0.0591, -0.0534, -0.0577,  ..., -0.0074, -0.0091, -0.0518]],
                     device='cuda:0')),
             ('fcl_stack.0.bias',
              tensor([-0.0693,  0.2537, -0.0491,  0.1794,  0.0732,  0.1356,  0.2742, -0.0847,
                       0.1632, -0.1957,  0.0778,  0.0637,  0.0085, -0.0930, -0.0127, -0.1241,
                       0.1139, -0.0599,  0.0155, -0.0106,  0.1650,  0.0669, -0.0662,  0.1834,
                      -0.1016,  0.0265,  0.1381, -0.0177,  0.0821,  0.0029, 

In [17]:
torch.save(model.state_dict(), 'model_params.pth')

In [20]:
model2 = NeuralNetwork().to(device) # NeuralNetwork.cuda()
print(model2) # model2는 훈련이 되지 않은 새로운 모델

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fcl_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [21]:
# 훈련되지 않은 모델로 검증하면 당연히 성능이 좋지 않다.
test_loop(test_dataloader, model2, loss_fn)

Test Error : 
 Accuracy : 15.6%, Avg Loss : 2.296766



In [22]:
params_file_path = "model_params.pth"

# 저장된 가중치로 새로운 모델에 덮어쓰기
model2.load_state_dict(torch.load(params_file_path))
test_loop(test_dataloader, model2, loss_fn)

C:\Users\user\AppData\Local\Temp\ipykernel_22608\856506580.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load(params_file_path))


Test Error : 
 Accuracy : 85.6%, Avg Loss : 0.413722



# 훈련된 모델 자체를 저장 / 불러오기
모델의 구조를 모르는 경우 사용할 수 있는 대표적인 방법으로서 가중치만 저장한 경우보다 파일의 크기는 크지만 구조를 몰라도 모델을 사용할 수 있다는 장점이 있습니다.

In [23]:
torch.save(model, 'model.pth')

In [24]:
!ls -al | grep pth

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [25]:
# weights_only -> 가중치만 가져오는 옵션
model3 = torch.load('model.pth', weights_only=False)
test_loop(test_dataloader, model3, loss_fn)

Test Error : 
 Accuracy : 85.6%, Avg Loss : 0.413722

